In [38]:
import pandas as pd
import json as js

with open("/u/pmpande/author_profiling/fb.json") as f:
    data_fb = pd.DataFrame(js.loads(line) for line in f)
f.close()

with open("/u/pmpande/author_profiling/essays.json") as f:
    data_essays = pd.DataFrame(js.loads(line) for line in f)
f.close()

print(len(data_fb))
print(len(data_essays))

9917
2469


In [39]:
data_fb.drop(data_fb.head(1).index, inplace=True)
data_essays.drop(data_essays.head(1).index, inplace=True)

In [40]:
count1 = 0
count2 = 0
count3 = 0
count4 = 0
count5 = 0
for row in data_fb.iterrows():
    if row[1]['AGR'] == 'y': count1 += 1
    if row[1]['CON'] == 'y': count2 += 1
    if row[1]['EXT'] == 'y': count3 += 1
    if row[1]['NEU'] == 'y': count4 += 1
    if row[1]['OPN'] == 'y': count5 += 1
print(str(count1/len(data_fb)))
print(str(count2/len(data_fb)))
print(str(count3/len(data_fb)))
print(str(count4/len(data_fb)))
print(str(count5/len(data_fb)))

0.5312626058894716
0.4594594594594595
0.4245663574021783
0.37474788221056876
0.7431423961274708


In [41]:
import random
rand_fb = random.sample(range(1,len(data_fb)-1),990) 
rand_essays = random.sample(range(1,len(data_essays)-1),250)

In [42]:
test1 = data_fb.iloc[rand_fb].copy()
train1 = data_fb.drop(rand_fb)

test2 = data_essays.iloc[rand_essays].copy()
train2 = data_essays.drop(rand_essays)

In [43]:
train1.drop(train1.columns[[1,7,8,9,10,11]], axis=1, inplace=True)
test1.drop(test1.columns[[1,7,8,9,10,11]], axis=1, inplace=True)

train2.drop(train2.columns[[1]], axis=1, inplace=True)
test2.drop(test2.columns[[1]], axis=1, inplace=True)

In [44]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

corpus1 = []
truth1 = []
for row in train1.iterrows():
    Y = [0,0,0,0,0]
    corpus1.append(row[1]['STATUS'])
    if row[1]['AGR'] == 'y': Y[0] = 1
    if row[1]['CON'] == 'y': Y[1] = 1
    if row[1]['EXT'] == 'y': Y[2] = 1
    if row[1]['NEU'] == 'y': Y[3] = 1
    if row[1]['OPN'] == 'y': Y[4] = 1
    truth1.append(Y)
truth1 = np.array(truth1)
corpus2 = []
truth2 = []
for row in train2.iterrows():
    Y = [0,0,0,0,0]
    corpus2.append(row[1]['STATUS'])
    if row[1]['AGR'] == 'y': Y[0] = 1
    if row[1]['CON'] == 'y': Y[1] = 1
    if row[1]['EXT'] == 'y': Y[2] = 1
    if row[1]['NEU'] == 'y': Y[3] = 1
    if row[1]['OPN'] == 'y': Y[4] = 1
    truth2.append(Y)
truth2 = np.array(truth2)

In [45]:
vectorizer1 = CountVectorizer(analyzer="word", token_pattern="[a-zA-Z]*", lowercase=True, stop_words="english")
x1 = vectorizer1.fit_transform(corpus1)
tfidf_transformer1 = TfidfTransformer()
X1 = tfidf_transformer1.fit_transform(x1)

vectorizer2 = CountVectorizer(analyzer="word", token_pattern="[a-zA-Z]*", lowercase=True, stop_words="english")
x2 = vectorizer2.fit_transform(corpus2)
tfidf_transformer2 = TfidfTransformer()
X2 = tfidf_transformer2.fit_transform(x2)

In [46]:
clf1 = DecisionTreeClassifier()
clf1_1 = RandomForestClassifier()
clf1 = clf1.fit(X1, truth1)
clf1_1 = clf1_1.fit(X1, truth1)

clf2 = DecisionTreeClassifier()
clf2_1 = RandomForestClassifier()
clf2 = clf2.fit(X2, truth2)
clf2_1 = clf2_1.fit(X2, truth2)

In [47]:
test_data1 = []
truth_test1 = []
for row in test1.iterrows():
    Y = [0,0,0,0,0]
    test_data1.append(row[1]['STATUS'])
    if row[1]['AGR'] == 'y': Y[0] = 1
    if row[1]['CON'] == 'y': Y[1] = 1
    if row[1]['EXT'] == 'y': Y[2] = 1
    if row[1]['NEU'] == 'y': Y[3] = 1
    if row[1]['OPN'] == 'y': Y[4] = 1
    truth_test1.append(Y)
truth_test1 = np.array(truth_test1)
test_data2 = []
truth_test2 = []
for row in test2.iterrows():
    Y = [0,0,0,0,0]
    test_data2.append(row[1]['STATUS'])
    if row[1]['AGR'] == 'y': Y[0] = 1
    if row[1]['CON'] == 'y': Y[1] = 1
    if row[1]['EXT'] == 'y': Y[2] = 1
    if row[1]['NEU'] == 'y': Y[3] = 1
    if row[1]['OPN'] == 'y': Y[4] = 1
    truth_test2.append(Y)
truth_test2 = np.array(truth_test2)
x_test1 = vectorizer1.transform(test_data1)
X_test1 = tfidf_transformer1.transform(x_test1)

x_test2 = vectorizer2.transform(test_data2)
X_test2 = tfidf_transformer2.transform(x_test2)

In [48]:
output1 = clf1.predict(X_test1)
output1_1 = clf1_1.predict(X_test1)
output2 = clf2.predict(X_test2)
output2_1 = clf2_1.predict(X_test2)

In [49]:
import numpy as np
from math import sqrt
from sklearn.metrics import accuracy_score, hamming_loss, mean_squared_error

print(accuracy_score(truth_test1, output1, normalize=True))
print(hamming_loss(truth_test1, output1))
print(sqrt(mean_squared_error(truth_test1, output1)))

print(accuracy_score(truth_test1, output1_1, normalize=True))
print(hamming_loss(truth_test1, output1_1))
print(sqrt(mean_squared_error(truth_test1, output1_1)))

0.888888888889
0.04606060606060606
0.21461734799546392
0.817171717172
0.06868686868686869
0.26208179770229884


In [50]:
print(accuracy_score(truth_test2, output2, normalize=True))
print(hamming_loss(truth_test2, output2))
print(sqrt(mean_squared_error(truth_test2, output2)))

print(accuracy_score(truth_test2, output2_1, normalize=True))
print(hamming_loss(truth_test2, output2_1))
print(sqrt(mean_squared_error(truth_test2, output2_1)))

0.932
0.032
0.17888543819998318
0.912
0.0384
0.19595917942265426
